
# AG2413 Digital Image Processing and Applications — **Lab 0**
Andrea Nascetti, Eric Brune  

> You will work in **Google Colab** or **Jupyter** using the **Earth Engine Python API** and **geemap** for interactive maps.



## Background: Google Earth Engine (GEE)
Google Earth Engine (GEE) is a cloud-based computing platform hosted by Google. GEE provides direct access to a multi-petabyte catalogue of satellite imagery and geospatial datasets and exposes **APIs for JavaScript and Python** to perform planetary-scale analysis of the Earth's surface.

**Main components of GEE**  
1) A petabyte-scale archive of publicly available remotely sensed imagery and other data.  
2) A computational infrastructure optimized for parallel processing of geospatial data.  
3) Application Programming Interfaces (APIs) for **JavaScript and Python** to make requests to the Earth Engine servers.



## Objective
This lab introduces the fundamentals needed to use the GEE **Python** API and guides you through several basic tasks.  
At the end of this lab, you will be able to use GEE to perform the following tasks:
- Run basic **Python** commands.
- Display and clip images.
- Explore image collections and their metadata.
- Filter image collections.
- Perform simple image band calculations (e.g., NDVI).
- Import (upload) and **export** images.



## Pre-lab requirements (complete **before** the lab starts)
- Create (or verify) a Google account (e.g., `abcdef@gmail.com`).
- Register for a GEE account at <https://earthengine.google.com/signup/>.
- **Python environment:** Use **Google Colab** (recommended) or local **Jupyter**.



---
# Part 0 — Python setup for Earth Engine & geemap
Run the next two cells **once at the top** of your notebook session (every new session in Colab).  
- The first cell installs required packages.  
- The second cell authenticates and initializes Earth Engine and sets up an interactive map.


In [1]:

# If running in Colab or a fresh Jupyter environment, install dependencies:
# (Re-run this cell at the start of each new session if packages are missing.)
%pip install --quiet earthengine-api geemap


Note: you may need to restart the kernel to use updated packages.


In [2]:

import ee, geemap

In [4]:


project_name = "kth-digital-image-processing"

# Authenticate on first use (opens a link in Colab). In local Jupyter, it will open a browser.
try:
    ee.Initialize(project = project_name)
    print("Earth Engine is already initialized.")
except Exception:
    ee.Authenticate()  # Follow the link and paste the code when prompted
    ee.Initialize(project = project_name)
    print("Earth Engine has been authenticated and initialized.")

# Create a map instance you can reuse throughout the lab
Map = geemap.Map()
Map


Earth Engine is already initialized.


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', transp…


---
# Part 1 — Getting started with Earth Engine in **Python**
For this course, we will use **Python** + **geemap** in a notebook:
- Code cells replace the Code Editor's Script panel.
- The **map widget** below each cell replaces the Code Editor's map window and layer manager.
- The notebook cell output replaces the Code Editor's **Console/Inspector**.
- There is **no Tasks tab** in a notebook; we manage exports via Python (`ee.batch.Export...`) and check status with `task.status()`.



## 1.2 Running your first Python cell
Execute the cell below.


In [5]:

print("Hello, World!")
a = 1
b = 2
print(f"{a} + {b} = {a+b}")


Hello, World!
1 + 2 = 3



## 1.3 Sharing your work
**Share your notebook**:
- **Colab:** File → Share → set to "Anyone with the link (Viewer)" and submit the link.
- **GitHub / Drive:** Commit the `.ipynb` or place it in Drive and share a view link.
- Include requested **screenshots** of map outputs (right-click the map output → Save image, or use your OS screenshot tool).



## 1.4 Python basics (translated from the JavaScript section)
A few quick translations from the original JavaScript notes to Python:

- Python ends statements with **newlines** (no semicolons).
- Create variables directly (no `var`).  
- Strings use single `'` or double `"` quotes.
- Lists use square brackets `[]` and are **zero-indexed** (first element is index `0`).
- Dictionaries (like JS objects) use curly braces `{}` with key-value pairs.
- Use `#` for comments; use triple quotes for multi-line strings if needed.
- Use functions with `def` and `return`.

> Note: On the Earth Engine server side, Python and JavaScript call **the same EE functions**. The algorithm semantics are the same; only client-side syntax differs.


In [6]:

# Lists, dicts, indexing
my_list = ['eggplant', 'apple', 'wheat']
print('First item:', my_list[0])

my_dict = {'food': 'bread', 'color': 'red', 'number': 42}
print('Color via key:', my_dict['color'])
print('Color via get():', my_dict.get('color'))

# Simple math
print('Subtracting two from three equals', 3-2)

# Functions
def my_hello_function(s: str) -> str:
    return f"Hello {s}!"

def add_function(a: int) -> int:
    return a + 3

print(my_hello_function('world'))
a = 1
print(f"{a} plus 3 is {add_function(a)}")


First item: eggplant
Color via key: red
Color via get(): red
Subtracting two from three equals 1
Hello world!
1 plus 3 is 4



## 1.5 Simple image visualization and analysis in Python
We'll load a **Landsat 5** TOA image and visualize it. We'll also create a Region of Interest (ROI) and clip to it.

**Tip:** `Map.addLayer(ee_object, vis_params, name)` in geemap mirrors the JS signature.


In [7]:

# Create a fresh map for this section
Map = geemap.Map()

# Landsat 5 TOA image
myimage = ee.Image('LANDSAT/LT05/C02/T1_TOA/LT05_015030_20100531')

# Syracuse city center
lon, lat = -76.147, 43.046
Map.setCenter(lon, lat, 10)

# Default display (not very informative)
Map.addLayer(myimage, {}, 'Layer 1 (default)')

# Color IR composite via explicit bands
vizParams = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0,
    'max': 0.5,
    'gamma': [0.95, 1.1, 1]
}
Map.addLayer(myimage, vizParams, 'Color IR composite 3')

# ROI: 20 km buffer around a point
roi = ee.Geometry.Point([lon, lat]).buffer(20000)
Map.addLayer(roi, {}, 'ROI')
Map.addLayer(myimage.clip(roi), vizParams, 'Color IR composite 4 (clipped)')

Map


Map(center=[43.046, -76.147], controls=(WidgetControl(options=['position', 'transparent_bg'], position='toprig…


### Part 1 – Task 1
- Include a **shareable link** to this notebook in your lab report.  
- Include a **screenshot** of your Landsat image (**Color IR composite 4**) from the map above.


In [8]:
## da Link no hinzuefuege

In [9]:
# Screenshot:

<img src="image1.png" alt="NDVI Difference Map" style="width:30%;">



## 1.6 Exploring image collections and metadata
An `ee.ImageCollection` is a stack (time series) of images. Below, we filter by bounds and date, and sort by cloud cover.


In [10]:

point = ee.Geometry.Point([lon, lat])
start = ee.Date('2014-06-01')
end   = ee.Date('2014-10-01')

filtered = (ee.ImageCollection('LANDSAT/LC08/C02/T1')
            .filterBounds(point)
            .filterDate(start, end)
            .sort('CLOUD_COVER', True))

first = ee.Image(filtered.first())

# Use getInfo() sparingly for small metadata/counts:
print('Number of images in filtered collection:', filtered.size().getInfo())
print('Least cloud cover for first image:', first.get('CLOUD_COVER').getInfo())

# Limit by WRS path/row and compute stats (Landsat 8 TOA)
collection = (ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA')
              .filter(ee.Filter.eq('WRS_PATH', 15))
              .filter(ee.Filter.eq('WRS_ROW', 30))
              .filterDate('2014-01-01', '2015-01-01'))

print('Count (path 15/row 30, 2014):', collection.size().getInfo())

sun_stats = collection.aggregate_stats('SUN_ELEVATION').getInfo()
print('Sun elevation stats:', sun_stats)

image = ee.Image(collection.sort('CLOUD_COVER').first())
print('Least cloudy image (ID):', image.get('LANDSAT_SCENE_ID').getInfo())

recent = collection.sort('system:time_start', False).limit(10)
print('Most recent 10 images (IDs):', recent.aggregate_array('LANDSAT_SCENE_ID').getInfo())


Number of images in filtered collection: 11
Least cloud cover for first image: 2.08
Count (path 15/row 30, 2014): 16
Sun elevation stats: {'max': 64.16618396, 'mean': 44.783931544999994, 'min': 21.2336852, 'sample_sd': 14.671322089897272, 'sample_var': 215.24769186550765, 'sum': 716.5429047199999, 'sum_sq': 35318.32377201854, 'total_count': 16, 'total_sd': 14.205446530254282, 'total_var': 201.79471112391343, 'valid_count': 16, 'weight_sum': 16, 'weighted_sum': 716.5429047199999}
Least cloudy image (ID): LC80150302014114LGN01
Most recent 10 images (IDs): ['LC80150302014354LGN02', 'LC80150302014338LGN01', 'LC80150302014322LGN01', 'LC80150302014306LGN01', 'LC80150302014290LGN01', 'LC80150302014274LGN01', 'LC80150302014258LGN01', 'LC80150302014242LGN01', 'LC80150302014226LGN01', 'LC80150302014210LGN01']



## 1.7 Performing image band calculations (NDVI difference)
Compute NDVI for two Landsat 5 scenes 20 years apart and visualize the difference.


In [11]:

def getNDVI(image):
    # Landsat 5: NIR=B4, Red=B3
    return image.normalizedDifference(['B4', 'B3']).rename('NDVI')

image1 = ee.Image('LANDSAT/LT05/C02/T1_TOA/LT05_015030_19880619')
image2 = ee.Image('LANDSAT/LT05/C02/T1_TOA/LT05_015030_20100531')

ndvi1 = getNDVI(image1)
ndvi2 = getNDVI(image2)
ndviDifference = ndvi2.subtract(ndvi1).rename('NDVI_Diff')

Map = geemap.Map()
Map.setCenter(lon, lat, 10)
Map.addLayer(ndviDifference, {'min': -1, 'max': 1}, 'NDVI difference')
Map


Map(center=[43.046, -76.147], controls=(WidgetControl(options=['position', 'transparent_bg'], position='toprig…


### Part 1 – Task 2
- Include a shareable link to your notebook.  
- **Select a region** where the NDVI difference between `image1` and `image2` is relatively large (bright).  
- Include a **screenshot** of that area (NDVI difference **and** Satellite basemap).  
- Identify the land cover and give a plausible explanation for the NDVI change.


In [12]:
# Task 2

# Link no inetue



### NDVI difference
<img src="image_diff.png" alt="NDVI Difference Map" style="width:30%;">

### Satellite basemap
<img src="image_sat.png" alt="NDVI Difference Map" style="width:30%;">

### Possible Explanation of High Difference

The parts of the map with a high NDVI difference appear to be agriculture land. It makes senses that more vegetation as grown between 1988 and 2010, resulting in the difference.


## 1.8 Exporting images (to Google Drive)
In notebooks we start an export task and check its status from Python.


In [13]:

# Example export of a Landsat 8 true-color image
landsat = (ee.Image('LANDSAT/LC08/C02/T1_TOA/LC08_123032_20140515')
           .select(['B4', 'B3', 'B2']))

geometry = ee.Geometry.Rectangle([116.2621, 39.8412, 116.4849, 40.01236])

task = ee.batch.Export.image.toDrive(
    image=landsat,
    description='imageToDriveExample',
    scale=30,
    region=geometry
)
task.start()
print('Export started. You can poll task.status() to see progress:')
print(task.status())


Export started. You can poll task.status() to see progress:
{'state': 'READY', 'description': 'imageToDriveExample', 'priority': 100, 'creation_timestamp_ms': 1762163878357, 'update_timestamp_ms': 1762163878357, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'YDLRAXVQIDJAZKWAXRMYG5PY', 'name': 'projects/kth-digital-image-processing/operations/YDLRAXVQIDJAZKWAXRMYG5PY'}



## 1.9 Importing (uploading) images to Earth Engine
To upload a GeoTIFF to your Earth Engine **Assets**, use one of the following:
- **Earth Engine Assets web UI:** In the Earth Engine **Code Editor** under the **Assets** tab, click **NEW → Image upload**, choose your GeoTIFF, and follow the prompts. The uploaded image will appear under your user assets.
- **(Optional) Command line:** The `earthengine` command-line tool supports uploads as well.

> After ingestion completes, your asset will be available to import in Python using its asset ID (e.g., `users/yourname/folder/your_asset`).



---
# Part 2 — Searching and Visualizing **Landsat 8** imagery
In this part, you'll search for a Landsat 8 scene over your **region of interest (ROI)** with few clouds and visualize it in multiple ways.



## 2.1 Searching (and finding) your own Landsat imagery
**a.** We'll work in this notebook cell rather than creating a separate JS file.  
**b.** Define your ROI: you can either **set coordinates** directly or use the map to pick a point and then paste its coordinates here.  
**c.** Import the 'USGS Landsat 8 Collection 1 Tier 1 TOA Reflectance' ImageCollection in Python by referencing its ID: `'LANDSAT/LC08/C01/T1_TOA'`.  
**d.** Filter by date/location, sort by `'CLOUD_COVER'`, and take the first image.


In [14]:

Map = geemap.Map()

# === TODO: REPLACE WITH YOUR OWN ROI COORDINATES ===
roi = ee.Geometry.Point([8.5417, 47.3769])  # Example: Stockholm city center

landsat = ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA')

image = (landsat
         .filterDate('2021-01-01', '2022-12-31')
         .filterBounds(roi)
         .sort('CLOUD_COVER')
         .first())

if image is None:
    raise ValueError("No image found for those filters. Try a wider date range or different ROI.")


print('A Landsat scene:', image) # prints a server-side object reference 
props = image.toDictionary() 
print('Cloud cover:', props.get('CLOUD_COVER').getInfo()) 
print('Acquisition date (DATE_ACQUIRED):', props.get('DATE_ACQUIRED').getInfo())

vis = {'bands': ['B4','B3','B2'], 'min': 0.03, 'max': 0.3}  

Map.centerObject(roi, 10)
Map.addLayer(roi, {}, 'ROI')
Map.addLayer(image, vis, 'L8 True Color')
Map.addLayerControl()

Map  

A Landsat scene: ee.Image({
  "functionInvocationValue": {
    "functionName": "Collection.first",
    "arguments": {
      "collection": {
        "functionInvocationValue": {
          "functionName": "Collection.limit",
          "arguments": {
            "collection": {
              "functionInvocationValue": {
                "functionName": "Collection.filter",
                "arguments": {
                  "collection": {
                    "functionInvocationValue": {
                      "functionName": "Collection.filter",
                      "arguments": {
                        "collection": {
                          "functionInvocationValue": {
                            "functionName": "ImageCollection.load",
                            "arguments": {
                              "id": {
                                "constantValue": "LANDSAT/LC08/C02/T1_TOA"
                              }
                            }
                          }
         

Map(center=[47.37689999999999, 8.541700000000002], controls=(WidgetControl(options=['position', 'transparent_b…


### Part 2 – Task 1
- Include a screenshot of your selected location from the map.  
- Report the **cloud cover** and **acquisition date** (`DATE_ACQUIRED`) of your scene (see printed output).


### Satellite Image of Zurich
<img src="zh.png" alt="ZH" style="width:30%;">

Since the best image of 2021 was still quite clouded over the city, the date range was expanded to 2022.
The cloud cover of the final image is 0.78.
It was acquired on 2022-07-16.


## 2.2 Visualizing Landsat 8 imagery
A **true-color** image uses bands mapped to R, G, B ≈ human visual perception. A **false-color** composite uses other bands (e.g., NIR) to emphasize features.

**a.** Add your image to the map as **true color**.


In [15]:

Map = geemap.Map()
Map.centerObject(roi, 10)

trueColor = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.3}
Map.addLayer(image, trueColor, 'True color')
Map


Map(center=[47.37689999999999, 8.541700000000002], controls=(WidgetControl(options=['position', 'transparent_b…


**b.** To select band names, consult the dataset's **BANDS** documentation in the Earth Engine Data Catalog.  
**c.** Use the Inspector (click on the map) or the layer stretch tool (in the layer manager widget) to estimate reasonable `min`/`max` display ranges.



### Part 2 – Task 2
Add a **false-color** visualization of your scene. Use **Near Infrared (NIR)**, **Red**, and **Green** as RGB inputs.

> **Fill in the correct band names** for NIR/Red/Green from the Landsat 8 TOA product page.

Also add **one more visualization** (your choice) optimized for a particular environment (e.g., water vs. forest). Include screenshots and briefly justify your choices.


In [16]:

falseColor = {
    'bands': ['B5', 'B4', 'B3'],
    'min': 0,
    'max': 0.3
}

Map = geemap.Map()
Map.centerObject(roi, 10)
Map.addLayer(image, falseColor, 'False color (NIR-Red-Green)')

# === OPTIONAL: Add another visualization tuned for a specific surface (e.g., water/forest) ===
# Example: SWIR-NIR-Red to highlight moisture/urban
customViz = {'bands': ['B6', 'B5', 'B4'], 'min': 0, 'max': 0.3}
Map.addLayer(image, customViz, 'Custom viz (example)')
Map


Map(center=[47.37689999999999, 8.541700000000002], controls=(WidgetControl(options=['position', 'transparent_b…

### False Color Image

The bands B5(NIR), B4(Red) and B3(Green) have to be selected for the false color image.

<img src="false_color.png" alt="ZH" style="width:30%;">

### Custom Visualization

The custom visualization highlights moist and urban areas:

<img src="custom_vis.png" alt="ZH" style="width:30%;">


---
# Part 3 — Searching and Visualizing **Sentinel-2 (Level-1C)** imagery
We will load the Stockholm County boundary, filter Sentinel-2 **L1C** images for summer 2021 with <10% clouds, build a median composite, and visualize several band combinations.



## 3.1 Study area (Stockholm County) and Sentinel-2 filtering


In [17]:

Map = geemap.Map()

# Stockholm County boundary from GAUL 2015 Level-1
assetId = 'FAO/GAUL_SIMPLIFIED_500m/2015/level1'
boundaries = ee.FeatureCollection(assetId)
stockholmCounty = (boundaries
                   .filter(ee.Filter.eq('ADM1_NAME', 'Stockholms Laen'))
                   .first()
                   .geometry())

Map.centerObject(stockholmCounty, 8)
Map.addLayer(stockholmCounty, {}, 'Stockholm County')

# Sentinel-2 MSI Level-1C collection
s2 = (ee.ImageCollection('COPERNICUS/S2')
      .filterDate('2021-06-01', '2021-08-31')
      .filterBounds(stockholmCounty)
      .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)))

print('Number of S2 scenes (summer 2021, <10% clouds):', s2.size().getInfo())

# Median composite clipped to county
composite = s2.median().clip(stockholmCounty)

# Visualizations: True color, Color IR, SWIR (SWIR2, NIR, Red)
true_color = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}
cir        = {'bands': ['B8', 'B4', 'B3'], 'min': 0, 'max': 3000}     # NIR, Red, Green
swir       = {'bands': ['B12', 'B8', 'B4'], 'min': 0, 'max': 3000}    # SWIR2, NIR, Red

Map.addLayer(composite, true_color, 'S2 True color (median)')
Map.addLayer(composite, cir,        'S2 Color IR (median)')
Map.addLayer(composite, swir,       'S2 SWIR (median)')
Map


Number of S2 scenes (summer 2021, <10% clouds): 60


Map(center=[59.46934665958029, 18.14619899396542], controls=(WidgetControl(options=['position', 'transparent_b…


### Part 3 – Task 1
- Report how many **Sentinel-2** scenes satisfy the criteria (see printed output).  
- Include **screenshots** of all three visualizations (True color, Color IR, SWIR).  
- Name at least one **land cover** type that is well distinguishable in each visualization and explain why.


60 Sentinel-2 scenes satisfy the criteria.

### Real Color

In the real color image, urban areas are distinguishable because of their grey tone.

<img src="real_color.png" alt="ZH" style="width:30%;">

### Color IR

In the color infrared image, vegetation is distinguishable, since vegetation strongly reflects near-infrared light.

<img src="ir.png" alt="ZH" style="width:30%;">

### SWIR composite

Water bodies are distinguishable in the SWIR image, since they absorb SWIR and appear very dark.

<img src="swir.png" alt="ZH" style="width:30%;">






---
## Notes & Tips
- Use `getInfo()` **sparingly** to pull small metadata or counts to the client; most operations should stay **server-side**.
- In notebooks there is no Tasks panel; check export status with `task.status()` and `task.active()`.
- If something fails with authentication, re-run the `ee.Authenticate()` / `ee.Initialize()` cell.
- For ROI creation via the map, you can draw a point/polygon for orientation and then paste the coordinates into your `roi` variable for reproducible code.
- Make sure all screenshots clearly show the **layer names** requested in each deliverable.




---
# Part 4 — Getting image pixels into **NumPy**

For small cutouts (a few hundred pixels on a side), you can bring pixel values from Earth Engine to the client and work with them as `numpy.ndarray`. Keep the total pixel count modest and prefer **server‑side** processing or **exports** for large areas.

We'll show two Python options:

- **Recommended (simple):** `geemap.ee_to_numpy()` – returns a NumPy (masked) array for a given region/scale/CRS.
- **Low‑level (Python‑only):** `ee.data.getPixels()` – directly fetches pixels; handy for very small chips. **If the chip is small enough, you can use this method.**



## 4.1 Convenience method: `geemap.ee_to_numpy` (recommended)

This pulls raw band values into a NumPy array for a small *chip* around a point.  
The array shape is `(rows, cols, bands)`.

- For **Sentinel‑2 L1C**, use `scale=10` for 10 m bands (e.g., B2/B3/B4/B8).
- For **Landsat 8/9 TOA**, use `scale=30`.

> Keep the chip small (e.g., ≤ 512×512) to avoid timeouts and large downloads.


In [18]:

import ee, geemap, numpy as np

# Choose a point (Stockholm by default) and the smallest-cloud Sentinel-2 image in summer 2021.
roi = ee.Geometry.Point([18.0686, 59.3293])

image = (ee.ImageCollection('COPERNICUS/S2')
         .filterBounds(roi)
         .filterDate('2021-06-01', '2021-08-31')
         .sort('CLOUDY_PIXEL_PERCENTAGE')
         .first()
         .select(['B4', 'B3', 'B2']))  # Red, Green, Blue

# Build a ~512x512 chip at 10 m (≈ 5.1 km across). Buffer radius = half the target width * scale.
chip = roi.buffer(256 * 10).bounds()

# Download to a NumPy masked array. Shape: (rows, cols, bands)
arr = geemap.ee_to_numpy(image, region=chip, scale=10)

# Convert to a standard ndarray with NaNs for masked pixels (optional)
arr_f32 = np.array(arr, dtype=np.float32)
if hasattr(arr, "mask"):
    arr_f32 = np.where(np.broadcast_to(~arr.mask[..., :1], arr.shape), arr_f32, np.nan)

print("NumPy array shape:", arr_f32.shape, "(rows, cols, bands)")
print("dtype:", arr_f32.dtype)
print("per-band min/max:", np.nanmin(arr_f32, axis=(0,1)), np.nanmax(arr_f32, axis=(0,1)))


NumPy array shape: (537, 535, 3) (rows, cols, bands)
dtype: float32
per-band min/max: [  1. 406.  72.] [24691. 12161. 11730.]



## 4.2 Low‑level method: `ee.data.getPixels`

`ee.data.getPixels` can return pixels directly as a **NumPy structured array** when `fileFormat='NUMPY_NDARRAY'`. This is fast and explicit, but you must define a grid (CRS, transform, width/height). Use it for **small** requests only.

The example below fetches a 512×512 chip (B4/B3/B2) centered on Stockholm. It converts the structured array to a standard `(H, W, 3)` NumPy array.


In [19]:

import ee, numpy as np


coords = [18.0686, 59.3293]  # Stockholm
pt = ee.Geometry.Point(coords)

# Least-cloud Sentinel-2 image in summer 2021 (RGB bands selected in B4,B3,B2 order)
image = (ee.ImageCollection('COPERNICUS/S2')
         .filterBounds(pt)
         .filterDate('2021-06-01', '2021-08-31')
         .sort('CLOUDY_PIXEL_PERCENTAGE')
         .first()
         .select(['B4', 'B3', 'B2']))

# Retrieve the asset ID needed by getPixels.
image_id = image.getInfo()['id']

# Build a projection in EPSG:4326 at ~10 m scale (degrees-per-pixel).
proj = ee.Projection('EPSG:4326').atScale(10).getInfo()
scale_x = proj['transform'][0]
scale_y = -proj['transform'][4]

# Request a NUMPY_NDARRAY (structured array) directly.
request = {
    'assetId': image_id,
    'fileFormat': 'NUMPY_NDARRAY',
    'bandIds': ['B4', 'B3', 'B2'],  # Red, Green, Blue
    'grid': {
        'dimensions': {'width': 512, 'height': 512},
        'affineTransform': {
            'scaleX': scale_x, 'shearX': 0, 'translateX': coords[0],
            'shearY': 0,       'scaleY': scale_y, 'translateY': coords[1],
        },
        'crsCode': proj['crs'],
    }
}

arr_struct = ee.data.getPixels(request)  # Structured ndarray with fields 'B4','B3','B2'

# Convert structured array (H,W) -> standard HxWxC float32 array
rgb = np.dstack([arr_struct['B4'], arr_struct['B3'], arr_struct['B2']]).astype(np.float32)

print("Structured fields:", arr_struct.dtype.names)
print("RGB array shape:", rgb.shape, "(H, W, 3)")
print("dtype:", rgb.dtype, "min/max:", np.nanmin(rgb), np.nanmax(rgb))


Structured fields: ('B4', 'B3', 'B2')
RGB array shape: (512, 512, 3) (H, W, 3)
dtype: float32 min/max: 1.0 24691.0



### Part 4 – Task 1 (optional)
- Use **either** method above to fetch a 256×256 RGB chip over your own ROI.
- Print the array **shape**, **dtype**, and **per‑band min/max**.
- Include a short note on when you would *avoid* client‑side pixel downloads and prefer server‑side operations or an export.
